In [1]:
#export
from exp.nb_12a import *

# Data

In [2]:
wiki = 'https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip'

In [3]:
path = untar_data(wiki)

In [4]:
Path.BASE_PATH = path

In [5]:
# !mv {path}/wiki.valid.tokens {path}/valid.txt
# !mv {path}/wiki.test.tokens {path}/test.txt
# !mv {path}/wiki.train.tokens {path}/train.txt

In [6]:
path.ls()

[Path('valid.txt'), Path('test.txt'), Path('train.txt')]

In [7]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

In [8]:
def read_wiki(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines) - 2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            current_article = current_article.replace('<unk>', UNK)
            articles.append(current_article)
            current_article = ''
    current_article = current_article.replace('<unk>', UNK)
    articles.append(current_article)
    return articles

In [9]:
articles = read_wiki(path/'train.txt')

In [10]:
len(articles)

28476

In [11]:
train = TextList(read_wiki(path/'train.txt'), path)
valid = TextList(read_wiki(path/'valid.txt'))

In [12]:
len(train), len(valid)

(28476, 60)

In [13]:
sd = SplitData(train, valid)

In [16]:
proc_tok, proc_num = TokenizeProcessor(max_workers=1), NumericalizeProcessor()

In [ ]:
ll = label_by_func(sd, lambda x: 0, proc_x=[proc_tok, proc_num])

In [ ]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [ ]:
ll = pickle.load(open(path/'ld.pkl', 'rb'))

In [ ]:
bs,bptt = 128,70
data = lm_databunchify(ll, bs, bptt)

In [ ]:
vocab = ll.train.proc_x[-1].vocab
len(vocab)

In [ ]:
ll.train.proc_x

# Model

In [ ]:
dps = np.array([0.1, 0.15, 0.25, 0.02, 0.2]) * 0.2
tok_pad = vocab.index(PAD)

In [ ]:
emb_sz, nh, nl = 300, 300, 2

In [ ]:
model = get_language_model(len(vocab), emb_sz, nh, nl, tok_pad, *dps)

In [ ]:
cbs = [partial(AvgStatsCallback, accuracy_flat),
      CudaCallback, Recorder, ProgressCallback,
      partial(GradientClipping, 0.1),
      partial(RNNTrainer, α=2., β=1.)]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [ ]:
lr = 5e-3
sched_lr = combine_scheds([0.3, 0.7], cos_1_cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds([0.3, 0.7], cos_1_cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [ ]:
learn.fit(2, cbs=cbsched)

In [ ]:
torch.save(learn.model.state_dict(), path/'pretrained.pth')
pickle.dump(vocab, open(path/'vocab.pkl', 'wb'))